In [1]:
# Notebook with ipywidget examples
#
# DVC - 24/11/2020
#
# Dependencies:
# - ipywidgets, Ipython.display, numpy
#
# Note: I'm using a code cell such that there is no output generated
#       Tried to add the nbconvert-hide tag, but that is not enough

# Introduction to ipywidgets
- overview of the basic widgets and stylings
- a simple guessing game putting as a demo app

### The most basic slider

In [2]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive
import IPython.display as ipd
import numpy as np

In [3]:
# widget1: integer slider
slider1 = widgets.IntSlider(max=20)
ipd.display(slider1)
def my_func(x):
    print(2*x)

IntSlider(value=0, max=20)

In [4]:
# The code below works as expected in Jupyter Notebook,
#    but not in Jupyter lab !, rendering in voila, ...
# button1 = widgets.Button(description="B1")
#
# def printMe(b):
#        print("Printing something")
# button1.on_click(printMe)
# button1

### A button  ... that prints

In [5]:
# simple button, but sending the output via output widget
button1b = widgets.Button(description="Click me to Print")
output1 = widgets.Output()
ipd.display(button1b,output1)
def printMe(b):
    with output1:
        print("Printed something")
button1b.on_click(printMe)
#button1b

Button(description='Click me to Print', style=ButtonStyle())

Output()

### A textbox  ... get's printed n times 

In [6]:
# widget 3: printing text
@interact( text='', times=(0,5) )
def printIt(text,times):
    print( text * times )

interactive(children=(Text(value='', description='text'), IntSlider(value=2, description='times', max=5), Outp…

### A progress bar ...  counts your clicks

In [7]:
progress_bar = widgets.IntProgress(
    value=0,
    min=0, max=10,step=1,
    description='Counting',
    bar_style='success',
    orientation='horizontal'
    )
btn_add = widgets.Button( description= 'add one ... ' )

def AddOne(b):
    progress_bar.value += 1
    if progress_bar.value == 10:
        progress_bar.bar_style=''
    elif progress_bar.value % 2 != 0:
        progress_bar.bar_style='danger'
    else:
        progress_bar.bar_style='success'
btn_add.on_click(AddOne)

widgets.HBox([btn_add, progress_bar])

### A dropdown menu ... doesn't do anything here

In [8]:
rec_play = widgets.Dropdown(
    options={'REC','PLAY'},
    description='ACTION',
)
rec_play

Dropdown(description='ACTION', options=('PLAY', 'REC'), value='PLAY')

### Linking widgets together with the Play widget

In [9]:
# widget2: animation by Play
play2 = widgets.Play(
    interval=1000, # interval in msec between animation frame
    value=50,
    min=0,
    max=100,
    step=1,
    description="Press play",
    disabled=False
)
slider2 = widgets.IntSlider()
widgets.jslink((play2, 'value'), (slider2, 'value'))
widgets.HBox([play2, slider2])

___
### Simple App: 
The outputs are displayed in the output widget and can be cleared as wished
___

In [10]:
# compose your app of a few controls and an output widget
# beautify by putting them all in boxes
def make_box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px'
     )

b1 = widgets.Button(description='button 1')
b2 = widgets.Button(description='button 2')
b3 = widgets.Button(description='CLEAR')
b3.style.button_color = 'red'

vbox1 = widgets.VBox([widgets.Label('Controls'), b1, b2])
vbox2 = widgets.VBox([widgets.Label('CTRL-CLEAR'), b3 ])
vbox1.layout = make_box_layout()
vbox2.layout = make_box_layout()

screen1 = widgets.Output()
screen1.layout = make_box_layout()
screen1.layout.width='200px'
screen2 = widgets.Output()
screen2.layout = make_box_layout()
screen2.layout.width='200px'

def b1_clicked(b):
    with screen1:
        print("Button 1 clicked.")
b1.on_click(b1_clicked)
def b2_clicked(b):
    with screen2:
        print("Button 2 clicked.")
b2.on_click(b2_clicked)
def b3_clicked(b):
    with screen1:
        ipd.clear_output()
    with screen2:
        ipd.clear_output()
b3.on_click(b3_clicked) 

myapp = widgets.HBox([vbox1, vbox2,screen1,screen2])
myapp

---
### GAME:  Beat the Bot (guessing game)
The bot learns from your previous clicking behavior   
Just try to beat him   
Have you found the optimum strategy

-----

In [11]:
# some globals as needed
global user_history
user_history = [1,0]
target_score = 5

#buttons
btn0 = widgets.Button( description= '0' )
btn1 = widgets.Button( description= '1' )
btnS = widgets.Button( description= 'START' )

def click_zero(b):
    update_game(0)
btn0.on_click( click_zero )
def click_one(b):
    update_game(1)
btn1.on_click( click_one )
def click_start(b):
    update_game(0,start=True)
btnS.on_click( click_start )
btn0.disabled = True
btn1.disabled = True
btn0.button_style ='info'
btn1.button_style ='info'
btnS.button_style ='warning'
# score board
user_score = widgets.IntProgress(
    value=0,
    min=0, max=target_score,step=1,
    description='You',
    bar_style='success',
    orientation='horizontal'
    )
bot_score = widgets.IntProgress(
    value=0,
    min=0, max=target_score,step=1,
    description='Bot',
    bar_style='danger',
    orientation='horizontal'
    )
scoreboard= widgets.VBox((user_score,bot_score))
final_msg = widgets.HTML("<h1 style='color:green'> You win </h1>")
final_msg.layout.visibility = "hidden"
# layout
game = widgets.VBox( ( widgets.HBox((scoreboard,final_msg)),
                   widgets.HBox((btn0,btn1,btnS))
             ))

In [12]:
#
# game rules
#
# note only user_history had to be defined as a global
# all widgets behave by nature as such

def update_game(user_choice,start=False):
    global user_history

    if( start ):
        user_history = [1,0]
        final_msg.value = "<h1 style='color:green'> You win </h1>"
        final_msg.layout.visibility = "hidden"
        user_score.value = 0
        bot_score.value = 0
        btn0.disabled = False
        btn1.disabled = False
        btnS.disabled = True
        btnS.button_style = ''
        return
    prob = sum(user_history)/len(user_history)
    comp_choice = np.random.binomial(1,prob,1)[0]
    user_history.append( user_choice )
    # print("sequence length: ",len(user_history)-2)
    if comp_choice == user_choice:
        bot_score.value += 1
    else:
        user_score.value += 1
    if user_score.value == target_score or bot_score.value == target_score:
        if bot_score.value == target_score:
            final_msg.value = "<h1 style='color:red'>Game Over</h1>"
        final_msg.layout.visibility = "visible"
        btn0.disabled = True
        btn1.disabled = True
        btnS.disabled = False
        btnS.button_style = 'warning'
    return

In [13]:
game